<a href="https://colab.research.google.com/github/sindhguvi/tamilcolloqhack/blob/main/tamilcolloq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install required libraries
!pip install transformers datasets torch

import os
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load dataset from Hugging Face
dataset = load_dataset("sindhujasan/tamilcolloq")  # Change 'your_username'

# Load pre-trained model and tokenizer (Use smaller model for faster training)
model_name = "facebook/blenderbot-small-90M"  # Use 90M model instead of 400M
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Tokenization function
def preprocess_function(examples):
    inputs = [f"{tamil} Meaning: {meaning}" for tamil, meaning in zip(examples["Tamil Word"], examples["Meaning"])]
    targets = examples["Usage in Tamil"]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Split into train & validation sets (fallback to 'train' if 'validation' is missing)
train_dataset = tokenized_datasets["train"]
try:
    test_dataset = tokenized_datasets["validation"]
except KeyError:
    print("Warning: 'validation' split not found. Using 'train' split for evaluation.")
    test_dataset = train_dataset  # Use train split for evaluation if validation is missing

# Optional: Use a **smaller** dataset for **faster testing**
small_train_dataset = train_dataset.select(range(100))  # First 100 samples
small_test_dataset = test_dataset.select(range(20))  # First 20 samples

# Training settings (Optimized for speed)
training_args = TrainingArguments(
    output_dir="./results",
    save_total_limit=2,         # Keep only the latest 2 checkpoints
    save_steps=500,             # Save model every 500 steps
    evaluation_strategy="steps",
    save_strategy="steps",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,         # Train for 1 epoch (change if needed)
    logging_steps=100,
    logging_dir="./logs",
    fp16=True,  # Enable 16-bit precision for faster GPU training
)

# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,  # Use small dataset for quick testing
    eval_dataset=small_test_dataset,
)

# Resume training from last checkpoint if available
last_checkpoint = "./results/checkpoint-last"
if os.path.exists(last_checkpoint):
    print("Resuming from last checkpoint...")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("Starting training from scratch...")
    trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_tamil_colloq_bot")
tokenizer.save_pretrained("./fine_tuned_tamil_colloq_bot")

# Load fine-tuned model for chatbot
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_tamil_colloq_bot")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_tamil_colloq_bot")

# Chatbot loop
print("Tamil Colloquial Chatbot is ready! Type 'exit' to end the chat.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break

    # Tokenize user input
    inputs = fine_tuned_tokenizer(user_input, return_tensors="pt").to(device)

    # Generate response
    reply_ids = fine_tuned_model.generate(**inputs)
    response = fine_tuned_tokenizer.decode(reply_ids[0], skip_special_tokens=True)

    print(f"Chatbot: {response}")


tokenizer_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/964k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/345k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: sindhuja-krisp (sindhuja-krisp-bharath-institute-of-higher-education-and) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,No log,9.119108
2,No log,8.131079
3,No log,7.950731


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 20, 'num_beams': 10, 'length_penalty': 0.65, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Exception: Error while initializing BPE: Token `_</w>` out of vocabulary